In [84]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        1
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [85]:
# we will verify that GPU is enabled for this notebook
# following should print: CUDA is available!  Training on GPU ...
#
# if it prints otherwise, then you need to enable GPU:
# from Menu > Runtime > Change Runtime Type > Hardware Accelerator > GPU

import torch
import numpy as np

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [86]:
import PIL
print(PIL.__version__)

9.5.0


In [87]:
!nvidia-smi
import torch
torch.cuda.is_available()

Fri Feb 16 12:43:13 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P0              29W /  70W |   2187MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

True

In [88]:
import pickle
import numpy as np
from skimage import io

from tqdm import tqdm, tqdm_notebook
from PIL import Image
from pathlib import Path

from torchvision import transforms
from multiprocessing.pool import ThreadPool
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import os
import torchvision as tv
from torchvision.transforms import v2
from matplotlib import colors, pyplot as plt
%matplotlib inline

# в sklearn не все гладко, чтобы в colab удобно выводить картинки
# мы будем игнорировать warnings
import warnings
warnings.filterwarnings(action='ignore', category=DeprecationWarning)


In [89]:
# разные режимы датасета
DATA_MODES = ['train', 'val', 'test']
# все изображения будут масштабированы к размеру 224x224 px
RESCALE_SIZE = 224
# работаем на видеокарте
DEVICE = torch.device("cuda")

In [90]:
list_of_simpsons = sorted(os.listdir('/kaggle/input/journey-to-springfield/train/simpsons_dataset'))

In [91]:
right_simpsons = dict.fromkeys(list_of_simpsons, 0)
total_simpsons = dict.fromkeys(list_of_simpsons, 0)
for hero in list_of_simpsons:
    right_simpsons[hero]+=1
    break
print(right_simpsons['abraham_grampa_simpson'])
#print("'"+'abraham_grampa_simpson')

1


In [92]:
#pip install -U albumentations

In [93]:
import albumentations as A
#from albumentations.pytorch.transforms import ToTensor
#from albumentations.pytorch import ToTensorV2
import cv2
import random

In [94]:
class SimpsonsDataset(Dataset):
    """
    Датасет с картинками, который паралельно подгружает их из папок
    производит скалирование и превращение в торчевые тензоры
    """
    def __init__(self, files, mode):
        super().__init__()
        # список файлов для загрузки
        self.files = sorted(files)
        # режим работы
        self.mode = mode

        if self.mode not in DATA_MODES:
            print(f"{self.mode} is not correct; correct modes: {DATA_MODES}")
            raise NameError

        self.len_ = len(self.files)

        self.label_encoder = LabelEncoder()

        if self.mode != 'test':
            self.labels = [path.parent.name for path in self.files]
            self.label_encoder.fit(self.labels)
            

            with open('label_encoder.pkl', 'wb') as le_dump_file:
                  pickle.dump(self.label_encoder, le_dump_file)

    def __len__(self):
        return self.len_

    def load_sample(self, file):
        image = Image.open(file)
        image.load()
        return image

    def __getitem__(self, index):
        # для преобразования изображений в тензоры PyTorch и нормализации входа
        x = self.load_sample(self.files[index])
        x = self._prepare_sample(x)
        x = np.array(x / 255, dtype='float32')
        #x = Image.fromarray(x)
        
        if self.mode == 'train':
            transform = v2.Compose([
                
                #A.RandomCrop(width=RESCALE_SIZE, height=RESCALE_SIZE),
                
                #A.RandomBrightnessContrast(p=0.3),
                v2.ToImage(),
                v2.ToDtype(torch.float32, scale=True),
                v2.ColorJitter(brightness=.3, contrast =0.3 ),#поподбирать данные
                v2. RandomRotation(degrees =(-10,10)),
                v2.RandomHorizontalFlip(p=0.5),
                v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
            x = transform(x)
            #x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
           
            
            
        else:
            transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
            x = transform(x)
            
            
        if self.mode == 'test':
            return x
        else:
            label = self.labels[index]
            label_id = self.label_encoder.transform([label])
            y = label_id.item()
            return x, y

    def _prepare_sample(self, image):
        image = image.resize((RESCALE_SIZE, RESCALE_SIZE))
        return np.array(image)

In [95]:
def imshow(inp, title=None, plt_ax=plt, default=False):
    """Imshow для тензоров"""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt_ax.imshow(inp)
    if title is not None:
        plt_ax.set_title(title)
    plt_ax.grid(False)

In [96]:
#определим директории с тренировочными и тестовыми файлами
train_dir = Path('/kaggle/input/journey-to-springfield/train/simpsons_dataset')
test_dir = Path('/kaggle/input/journey-to-springfield/testset')

In [97]:
train_val_files = sorted(list(train_dir.rglob('*.jpg')))
test_files = sorted(list(test_dir.rglob('*.jpg')))

In [98]:
from sklearn.model_selection import train_test_split

train_val_labels = [path.parent.name for path in train_val_files]
train_files, val_files = train_test_split(train_val_files, test_size=0.25, \
                                          stratify=train_val_labels)

In [99]:
random.choice([RESCALE_SIZE, 100])

224

In [ ]:
val_dataset = SimpsonsDataset(val_files, mode='val')
tr_dataset = SimpsonsDataset(train_files, mode='train')
fig, ax = plt.subplots(nrows=3, ncols=3,figsize=(8, 8), \
                        sharey=True, sharex=True)
for fig_x in ax.flatten():
    random_characters = int(np.random.uniform(0,10))
    im_val, label = tr_dataset[random_characters]
    img_label = " ".join(map(lambda x: x.capitalize(),\
                tr_dataset.label_encoder.inverse_transform([label])[0].split('_')))
    imshow(im_val.data.cpu(), \
          title=img_label,plt_ax=fig_x)

In [101]:
model = tv.models.resnet18(pretrained = True)
model.fc = nn.Linear(512,42)
model = model.to(DEVICE)

In [102]:
from torch.serialization import save

In [103]:
def fit_epoch(model, train_loader, criterion, optimizer):
    running_loss = 0.0
    running_corrects = 0
    processed_data = 0

    for inputs, labels in train_loader:
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        preds = torch.argmax(outputs, 1)
        
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        processed_data += inputs.size(0)
        
    train_loss = running_loss / processed_data
    train_acc = running_corrects.cpu().numpy() / processed_data
    return train_loss, train_acc

In [104]:
def eval_epoch(model, val_loader, criterion):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    processed_size = 0

    for inputs, labels in val_loader:
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)

        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            preds = torch.argmax(outputs, 1)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        processed_size += inputs.size(0)
    val_loss = running_loss / processed_size
    val_acc = running_corrects.double() / processed_size
    return val_loss, val_acc

In [105]:
def train(train_files, val_files, model, optimizer, criterion, scheduler,  epochs, batch_size):
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    history = []
    log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} \
    val_loss {v_loss:0.4f} train_acc {t_acc:0.4f} val_acc {v_acc:0.4f}"

    with tqdm(desc="epoch", total=epochs) as pbar_outer:
        opt = optimizer

        for epoch in range(epochs):
            train_loss, train_acc = fit_epoch(model, train_loader, criterion, opt)
            
            if scheduler is not None:
                scheduler.step()
                
            print("loss", train_loss)

            val_loss, val_acc = eval_epoch(model, val_loader, criterion)
            history.append((train_loss, train_acc, val_loss, val_acc))

            pbar_outer.update(1)
            tqdm.write(log_template.format(ep=epoch+1, t_loss=train_loss,\
                                           v_loss=val_loss, t_acc=train_acc, v_acc=val_acc))

    return history

In [106]:
def predict(model, test_loader):
    with torch.no_grad():
        logits = []

        for inputs in test_loader:
            inputs = inputs.to(DEVICE)
            model.eval()
            outputs = model(inputs).cpu()
            logits.append(outputs)

    probs = nn.functional.softmax(torch.cat(logits), dim=-1).numpy()
    return probs

In [107]:
def save_model(model, criterion, optimizer, scheduler, file='/kaggle/working/resnet_simpsons.pt'):
    torch.save({
        'model_state_dict': model.state_dict(),
        'criterion_state_dict': criterion.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict()
    }, file)

In [108]:
def load_resnet18(criterion, optimizer, scheduler, outputs, file='/kaggle/working/resnet_simpsons.pt', device=DEVICE):
    checkpoint = torch.load(file, map_location=device)

    model = tv.models.resnet18(pretrained=True)
    model.fc = nn.Linear(512, outputs)
    model = model.to(device)

    model.load_state_dict(checkpoint['model_state_dict'])
    criterion.load_state_dict(checkpoint['criterion_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    return model, criterion, optimizer, scheduler

In [109]:
if val_dataset is None:
    val_dataset = SimpsonsDataset(val_files, mode='val')

train_dataset = SimpsonsDataset(train_files, mode='train')

In [110]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()
        
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.05)

In [135]:
model, crit, opt, sch = load_resnet18(criterion, optimizer, scheduler,outputs = 42)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [136]:
history = train(train_dataset, val_dataset, model=model, optimizer=opt, criterion=crit, scheduler=sch, epochs=1, batch_size=64)

epoch:   0%|          | 0/1 [00:00<?, ?it/s]

loss 0.21528680978140122


epoch: 100%|██████████| 1/1 [04:17<00:00, 257.08s/it]


Epoch 001 train_loss: 0.2153     val_loss 0.2614 train_acc 0.9799 val_acc 0.9589


In [ ]:
save_model(model, criterion, optimizer, scheduler)

In [ ]:
label_encoder = pickle.load(open("label_encoder.pkl", 'rb'))

In [ ]:
m = {i: [] for i in range(42)}
p = {i: [] for i in range(42)}

k = 0
for i in range(len(tr_dataset)):
  if k % 2 == 0:
    im_val, label = tr_dataset[i]
    img_label = " ".join(map(lambda x: x.capitalize(),\
                  tr_dataset.label_encoder.inverse_transform([label])[0].split('_')))
    prob_pred = predict_one_sample(model, im_val.unsqueeze(0))
    predicted_proba = np.max(prob_pred)*100
    y_pred = np.argmax(prob_pred)
    pred_img_label = " ".join(map(lambda x: x.capitalize(),\
                  tr_dataset.label_encoder.inverse_transform([y_pred])[0].split('_')))
    if y_pred != label:
      p[label].append(0)
    else:
      p[label].append(1)
  k += 1

In [ ]:
print("N", "\t","Сharacter name", (40 - len(list_of_simpsons)) * " ", " Acc", (20 - len(str(k))) * " ", "\t\tnumber of images")
for i in range(len(p)):
    m[i].append(sum(p[i]) / len(p[i]))
    s = " ".join(map(lambda x: x.capitalize(),\
                  tr_dataset.label_encoder.inverse_transform([i])[0].split('_')))
    k = sum(p[i]) / len(p[i])
    print(i, "\t",s, (40 - len(s)) * " ", k, (20 - len(str(k))) * " ", len(p[i]))

N 	 Сharacter name   Acc                 		number of images
0 	 Abraham Grampa Simpson                    1.0                   343
1 	 Agnes Skinner                             0.8125                16
2 	 Apu Nahasapeemapetilon                    0.9957081545064378    233
3 	 Barney Gumble                             0.95                  40
4 	 Bart Simpson                              0.9900596421471173    503
5 	 Carl Carlson                              1.0                   37
6 	 Charles Montgomery Burns                  0.9888392857142857    448
7 	 Chief Wiggum                              0.991869918699187     369
8 	 Cletus Spuckler                           0.9444444444444444    18
9 	 Comic Book Guy                            0.9886363636363636    176
10 	 Disco Stu                                 0.6666666666666666    3
11 	 Edna Krabappel                            0.9941520467836257    171
12 	 Fat Tony                                  0.9                   10
13 	 Gil

In [ ]:
def predict_one_sample(model, inputs, device=DEVICE):
    """Предсказание, для одной картинки"""
    with torch.no_grad():
        inputs = inputs.to(device)
        model.eval()
        logit = model(inputs).cpu()
        probs = torch.nn.functional.softmax(logit, dim=-1).numpy()
    return probs

In [134]:
def predict(model, test_loader):
    with torch.no_grad():
        logits = []

        for inputs in test_loader:
            inputs = inputs.to(DEVICE)
            model.eval()
            outputs = model(inputs).cpu()
            logits.append(outputs)

    probs = nn.functional.softmax(torch.cat(logits), dim=-1).numpy()
    return probs

In [ ]:
#idxs = list(map(int, np.random.uniform(0,1000, 20)))
#imgs = [tr_dataset[id][0].unsqueeze(0) for id in idxs]

#probs_ims = predict(model, imgs)

In [131]:
sasha_dir = Path('/kaggle/input/artem23')
sasha_file = sorted(list(sasha_dir.rglob('*.jpg')))
#print(sasha_file)
sasha_dataset = SimpsonsDataset(sasha_file, mode='test')
#print(sasha_dataset[0])

In [132]:
prob_pred = predict_one_sample(model, sasha_dataset[0].unsqueeze(0))

In [133]:
predicted_proba = np.max(prob_pred)*100
print(predicted_proba)
y_pred = np.argmax(prob_pred)
predicted_label = label_encoder.classes_[y_pred]
print(predicted_label)

52.1084725856781
mayor_quimby


In [ ]:
y_pred = np.argmax(probs_ims,-1)

actual_labels = [val_dataset[id][1] for id in idxs]

preds_class = [label_encoder.classes_[i] for i in y_pred]

NameError: name 'probs_ims' is not defined

In [ ]:
print(list_of_simpsons[9])

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
f1 = f1_score(actual_labels, y_pred, average='micro')
report =classification_report(actual_labels, y_pred, output_dict=True, zero_division=0)
#print(report['macro avg']['precision'] )
print(report)
#print("F1-оценка:", f1)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(true_labels, predictions))

In [ ]:
import matplotlib.patches as patches
from matplotlib.font_manager import FontProperties

fig, ax = plt.subplots(nrows=3, ncols=3,figsize=(12, 12), \
                        sharey=True, sharex=True)
for fig_x in ax.flatten():
    random_characters = int(np.random.uniform(0,1000))
    im_val, label = val_dataset[random_characters]
    img_label = " ".join(map(lambda x: x.capitalize(),\
                val_dataset.label_encoder.inverse_transform([label])[0].split('_')))



    imshow(im_val.data.cpu(), \
          title=img_label,plt_ax=fig_x)

    actual_text = "Actual : {}".format(img_label)

    fig_x.add_patch(patches.Rectangle((0, 53),86,35,color='white'))
    font0 = FontProperties()
    font = font0.copy()
    font.set_family("fantasy")
    prob_pred = predict_one_sample(resnet, im_val.unsqueeze(0))
    predicted_proba = np.max(prob_pred)*100
    y_pred = np.argmax(prob_pred)

    predicted_label = label_encoder.classes_[y_pred]
    predicted_label = predicted_label[:len(predicted_label)//2] + '\n' + predicted_label[len(predicted_label)//2:]
    predicted_text = "{} : {:.0f}%".format(predicted_label,predicted_proba)

    fig_x.text(1, 59, predicted_text , horizontalalignment='left', fontproperties=font,
                    verticalalignment='top',fontsize=8, color='black',fontweight='bold')

In [124]:
test_dataset = SimpsonsDataset(test_files, mode="test")
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=64)
probs = predict(model, test_loader)


preds = label_encoder.inverse_transform(np.argmax(probs, axis=1))
test_filenames = [path.name for path in test_dataset.files]


In [125]:
import pandas as pd
#my_submit = pd.read_csv("/kaggle/working/submission.csv",  delim_whitespace=True)
my_submit = pd.DataFrame({'Image_id': test_filenames, 'Expected': preds})

my_submit.head()

,Image_id,Expected
0,img0.jpg,nelson_muntz
1,img1.jpg,bart_simpson
2,img10.jpg,ned_flanders
3,img100.jpg,chief_wiggum
4,img101.jpg,apu_nahasapeemapetilon


In [127]:
my_submit.to_csv('simpsons_resnet.csv', index=False, header = 1)